In [ ]:
# import all packages needed

import os, sys
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

## Separate target data by ethnicity (ABCD)

In [ ]:
# Basic setting

path = '/mnt/db2/genome/ABCD'
target = 'ABCD_release_3.0_QCed' # target file name
demographic = '/mnt/db2/genome/ABCD/info/acspsw03.txt'
pick = 'Others' # Choose White/Black/Hispanic/Asian/Others
savepath = '/mnt/db2/genome/ABCD/Input/Others'

PLINK = '/mnt/db2/Plink' # PLINK path

In [ ]:
def SepEth(target, demographic, pick, savepath):
    
    # Set the File
    target = pd.read_csv(path + '/' + target + '.fam', delimiter = '\t', header = None)
    demo = pd.read_csv(demographic, delimiter = '\t')
    
    # Change type of the 'race_ethnicity' column in the demographic
    demo = demo[['subjectkey', 'race_ethnicity']]
    
    # Merge two data
    ethnicity = pd.merge(target, demo, left_on = 1, right_on = 'subjectkey')
    ethnicity = ethnicity.drop_duplicates(['subjectkey'], keep = 'first')
    ethnicity = ethnicity.dropna(axis=0)
    ethnicity['race_ethnicity'] = ethnicity['race_ethnicity'].astype(int)
    
    # Separate ethnicity
    if (pick == 'White'):
        data = ethnicity[(ethnicity['race_ethnicity'] == 1)]
    elif (pick == 'Black'):
        data = ethnicity[(ethnicity['race_ethnicity'] == 2)]
    elif (pick == 'Hispanic'):
        data = ethnicity[(ethnicity['race_ethnicity'] == 3)]
    elif (pick == 'Asian'):
        data = ethnicity[(ethnicity['race_ethnicity'] == 4)]
    elif (pick == 'Others'):
        data = ethnicity[(ethnicity['race_ethnicity'] == 5)]

    # Save file
    data = data.iloc[:, :2]
    filename = 'ABCD_' + pick
    data.to_csv(savepath + '/' + filename + '.txt', sep = ' ', header = None, index = False)

In [ ]:
# Make ethnicity file

SepEth(target, demographic, pick, savepath)

In [ ]:
# Make new binary file for ABCD separate file!

os.chdir(PLINK)

os.system('./plink --bfile {} --keep {} --make-bed --out {}'.format(path + '/' + target, savepath + '/' + 'ABCD_' + pick + '.txt', savepath + '/' + 'ABCD_' + pick))

# Quality control

In [ ]:
config = {
    'Filepath' : None,
    'Filename' : None,
    'Bad_batch_file' : None,
    'Phenotype_file' : None,
    'Genotype_miss' : 0.05,
    'Individual_miss' : 0.05,
    'Minor allele frequency' : 0.01,
    'Chromosome' : [1,22],
    'Hardy weinberg equil' : 1e-6,
    'Prune' : {
        'window_size': 200,
        'sliding': 50,
        'r2': 0.25,
    },
    'Heterozygosity(SD)' : 3,
    'Relatedness' : {
        'min' : None,
        'max' : None,
    },
}

In [ ]:
Filepath = '/mnt/db2/genome/ABCD/Input/Others'
Filename = 'ABCD_Others'

PLINK = '/mnt/db2/Plink' # PLINK path

Bad_batch_file = 'batch_bad.txt'

Phenotype_file = None #'qtrait.txt'

# Set the thresholds of --geno, --mind, --maf --hwe
Genotype_miss = None
Individual_miss = None
MAF = None
HWE = None

Chromosome = None #[,]

Prune_window_size = None
Prune_sliding = None
Prune_r2 = None

Heterozygosity_SD = 4

Relatedness_min = 0.4
Relatedness_max = None

In [ ]:
#Config
config['Filepath'] = Filepath
config['Filename'] = Filename
config['Bad_batch_file'] = Bad_batch_file
config['Genotype_miss'] = config['Genotype_miss'] if Genotype_miss == None else Genotype_miss
config['Individual_miss'] = config['Individual_miss'] if Individual_miss == None else Individual_miss
config['Minor allele frequency'] = config['Minor allele frequency'] if MAF == None else MAF
config['Chromosome'] = config['Chromosome'] if Chromosome == None else Chromosome
config['Prune']['window_size'] = config['Prune']['window_size'] if Prune_window_size == None else Prune_window_size
config['Prune']['sliding'] = config['Prune']['sliding'] if Prune_sliding == None else Prune_sliding
config['Prune']['r2'] = config['Prune']['r2'] if Prune_r2 == None else Prune_r2
config['Heterozygosity(SD)'] = config['Heterozygosity(SD)'] if Heterozygosity_SD == None else Heterozygosity_SD
config['Relatedness']['min'] = Relatedness_min
config['Relatedness']['max'] = Relatedness_max

In [ ]:
# Make all directory

os.mkdir(config['Filepath'] + '/' + "Final_results")
os.mkdir(config['Filepath'] + '/' + '1.Bad_batch_removed')
os.mkdir(config['Filepath'] + '/' + '2.Missing_genotype')
os.mkdir(config['Filepath'] + '/' + '3.Missing_individuals')
os.mkdir(config['Filepath'] + '/' + '4.Impute_sex')
os.mkdir(config['Filepath'] + '/' + '5.Choose_chromosome')
os.mkdir(config['Filepath'] + '/' + '6.Minor_allele_frequency')
os.mkdir(config['Filepath'] + '/' + '7.HWE')
os.mkdir(config['Filepath'] + '/' + '8.Pruning')
os.mkdir(config['Filepath'] + '/' + '9.Heterozygosity')
os.mkdir(config['Filepath'] + '/' + '10.Relatedness')

In [ ]:
# Change direcctory which PLINK software is in
os.chdir(PLINK)

#Save config
with open(config['Filepath'] + '/' + 'Final_results/Used_parameters.json', 'w') as f:
        json.dump(config, f,indent = 0)

#Exclude bad batch
if config['Bad_batch_file'] != None:
    print('Bad batch file detected! Processing...')
    os.system('./plink --bfile {} --remove {} --out {}/1.Bad_batch_removed/ABCD_excluded --make-bed'.format(config['Filepath'] + '/' +  config['Filename'], config['Filepath'] + '/' + config['Bad_batch_file'], config['Filepath']))
else:
    print('No bad batch file detected. Anyway, New file will be made!')
    os.system('./plink --bfile {} --out {}/1.Bad_batch_removed/ABCD_excluded --make-bed'.format(config['Filepath'] + '/' + config['Filename'], config['Bad_batch_file'], config['Filepath']))

#Exclude missing genotype
os.system('./plink --bfile {}/1.Bad_batch_removed/ABCD_excluded --geno {} --out {}/2.Missing_genotype/ABCD_geno --make-bed'.format(config['Filepath'], config['Genotype_miss'], config['Filepath']))

#Exclude missing individuals
os.system('./plink --bfile {}/2.Missing_genotype/ABCD_geno --mind {} --out {}/3.Missing_individuals/ABCD_mind --make-bed'.format(config['Filepath'], config['Individual_miss'], config['Filepath']))

#Impute sex
os.system('./plink --bfile {}/3.Missing_individuals/ABCD_mind --impute-sex --out {}/4.Impute_sex/ABCD_sex --make-bed'.format(config['Filepath'], config['Filepath']))

#Choose which chromosome to use
os.system('./plink --bfile {}/4.Impute_sex/ABCD_sex --chr {}-{} --out {}/5.Choose_chromosome/ABCD_chr --make-bed'.format(config['Filepath'], config['Chromosome'][0],
                                                                                                                  config['Chromosome'][1], config['Filepath']))

#Minor Allele Frequency
os.system('./plink --bfile {}/5.Choose_chromosome/ABCD_chr --maf {} --out {}/6.Minor_allele_frequency/ABCD_maf --make-bed'.format(config['Filepath'], config['Minor allele frequency'], config['Filepath']))

#Hardy Weinberg Equilibrium
os.system('./plink --bfile {}/6.Minor_allele_frequency/ABCD_maf --hwe {} --out {}/7.HWE/ABCD_hwe --make-bed'.format(config['Filepath'], config['Hardy weinberg equil'], config['Filepath']))

#Pruning
os.system('./plink --bfile {}/7.HWE/ABCD_hwe --indep-pairwise {} {} {} --out {}/8.Pruning/ABCD_pruned'.format(config['Filepath'], config['Prune']['window_size'],
                                                                                       config['Prune']['sliding'],
                                                                                       config['Prune']['r2'], config['Filepath']))

#Make pruned dataset
os.system('./plink --bfile {}/7.HWE/ABCD_hwe --extract {}/8.Pruning/ABCD_pruned.prune.in --out {}/8.Pruning/ABCD_pruned --make-bed'.format(config['Filepath'], config['Filepath'], config['Filepath']))

#Heterozygosity with pruned dataset
os.system('./plink --bfile {}/8.Pruning/ABCD_pruned --het --out {}/9.Heterozygosity/ABCD_het'.format(config['Filepath'], config['Filepath']))

#Make removal list with heterozygosity calculation
ABCD_het = pd.read_csv(config['Filepath'] + '/' + '9.Heterozygosity/ABCD_het.het', sep = ' ', low_memory = False,
                       skipinitialspace=True)

#Calculating heterozygosity
ABCD_het['het'] = (ABCD_het['N(NM)']-ABCD_het['O(HOM)'])/ABCD_het['N(NM)']
mean = np.mean(ABCD_het['het'])
std = np.std(ABCD_het['het'])

over_out = ABCD_het.loc[ABCD_het['het'] > mean+int('{}'.format(config['Heterozygosity(SD)']))*std,:]
under_out = ABCD_het.loc[ABCD_het['het'] < mean-int('{}'.format(config['Heterozygosity(SD)']))*std, :]
remove = pd.concat([over_out,under_out], axis=0)
remove.to_csv(config['Filepath'] + '/' + '9.Heterozygosity' +'/' + 'het_remove.txt', sep = ' ', header=None, index = False)

os.system('./plink --bfile {}/7.HWE/ABCD_hwe --remove {}/9.Heterozygosity/het_remove.txt --out {}/9.Heterozygosity/ABCD_het_removed --make-bed'.format(config['Filepath'], config['Filepath'], config['Filepath']))

if config['Relatedness']['min'] != None and config['Relatedness']['max'] == None:
    os.system('./plink --bfile {}/8.Pruning/ABCD_pruned --genome --min {} --out {}/10.Relatedness/ABCD_genome'.format(config['Filepath'], config['Relatedness']['min'], config['Filepath']))
if config['Relatedness']['min'] == None and config['Relatedness']['max'] != None:
    os.system('./plink --bfile {}/8.Pruning/ABCD_pruned --genome --max {} --out {}/10.Relatedness/ABCD_genome'.format(config['Filepath'], config['Relatedness']['max'], config['Filepath']))
if config['Relatedness']['min'] != None and config['Relatedness']['max'] != None:
    os.system('./plink --bfile {}/8.Pruning/ABCD_pruned --genome --min {} --max {} --out {}/10.Relatedness/ABCD_genome'.format(config['Filepath'], config['Relatedness']['min'],
                                                                                                         config['Relatedness']['max'], config['Filepath']))
if config['Relatedness']['min'] == None and config['Relatedness']['max'] == None:
    os.system('./plink --bfile {}/8.Pruning/ABCD_pruned --genome --out {}/10.Relatedness/ABCD_genome'.format(config['Filepath'], config['Filepath']))

ABCD_genome = pd.read_csv(config['Filepath'] + '/' + '10.Relatedness/ABCD_genome.genome', sep = ' ', low_memory = False,
                       skipinitialspace=True)

#get missing rate information
os.system('./plink --bfile {}/1.Bad_batch_removed/ABCD_excluded --missing --out {}/10.Relatedness/ABCD_miss'.format(config['Filepath'], config['Filepath']))
miss = pd.read_csv(config['Filepath'] + '/' + '10.Relatedness/ABCD_miss.imiss', sep = ' ', low_memory = False,
                       skipinitialspace=True)

#Compare individuals missing rate and leave only good one for each pairs
low_call_list = []
low_call_val = []
for i in range(ABCD_genome.shape[0]):
    f1 = ABCD_genome.loc[i,'FID1']
    p1 = ABCD_genome.loc[i,'IID1']
    f2 = ABCD_genome.loc[i,'FID2']
    p2 = ABCD_genome.loc[i,'IID2']
    c1 = float(miss.loc[miss['IID'] == p1, 'F_MISS'])
    c2 = float(miss.loc[miss['IID'] == p2, 'F_MISS'])
    if c1>c2:
        low_call_list.append([f1,p1])
        low_call_val.append([c1,c2])
    elif c2>c1:
        low_call_list.append([f2,p2])
        low_call_val.append([c1,c2])
    else: continue
        
#Save low call individual list
f = open(config['Filepath'] + '/' + '10.Relatedness' + '/' + "low_call.txt", 'w')
for i in range(len(low_call_list)):
    a1 = str(low_call_list[i]).replace("['",'')
    a2 = a1.replace("']",'')
    a3 = a2.replace("', '",' ')
    data = a3 + '\n'
    f.write(data)
f.close()

#Exclude low call list from data
os.system('./plink --bfile {}/9.Heterozygosity/ABCD_het_removed --remove {}/10.Relatedness/low_call.txt --out {}/Final_results/ABCD_QC_finished --make-bed'.format(config['Filepath'], config['Filepath'], config['Filepath']))